In [9]:
!pip install transformers
!pip install sentencepiece
!pip install openpyxl

/bin/bash: /home/jstil/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 4.7 MB/s eta 0:00:00a 0:00:01


In [2]:
!nvidia-smi

/bin/bash: /home/jstil/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Fri Oct 28 14:02:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 522.25       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| N/A   60C    P8    16W /  N/A |    335MiB /  8192MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+-----------------

In [14]:
import sys
import os
import time
import re
import random
from typing import Dict, List, Optional, Union
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import tensorflow as tf
from transformers import BertTokenizer, BertConfig, TFBertForSequenceClassification
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from transformers import ElectraTokenizer, TFElectraForSequenceClassification
from transformers import XLNetTokenizer, TFXLNetForSequenceClassification
from transformers import LongformerTokenizer, TFLongformerForSequenceClassification
from transformers import DebertaTokenizer, TFDebertaForSequenceClassification

In [15]:
# set seed, TF uses python ramdom and numpy library, so these must also be fixed
tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)
os.environ['PYTHONHASHSEED']=str(0)
os.environ['TF_DETERMINISTIC_OPS'] = '0'

In [5]:
# see if hardware accelerator available
tf.config.experimental.list_physical_devices()

2022-10-28 14:02:59.984529: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-28 14:03:00.063974: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-28 14:03:00.064611: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
tf.test.gpu_device_name()

2022-10-28 14:03:18.973017: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-28 14:03:18.978657: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-28 14:03:18.979225: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-28 14:03:18.979709: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

'/device:GPU:0'

In [18]:
PATH_sg1 = "data/final_labels_SG1.xlsx"
PATH_sg2 = "data/final_labels_SG2.xlsx"
df_sg1 = pd.read_excel(PATH_sg1)
df_sg2 = pd.read_excel(PATH_sg2)
df_sg1.rename(columns={'text': 'sentence', 'label_bias': 'Label_bias'}, inplace=True)
df_sg2.rename(columns={'text': 'sentence', 'label_bias': 'Label_bias'}, inplace=True)
df_sg1.head()

,sentence,news_link,outlet,topic,type,Label_bias,label_opinion,biased_words
0,The Republican president assumed he was helpin...,http://www.msnbc.com/rachel-maddow-show/auto-i...,msnbc,environment,left,Biased,Expresses writer’s opinion,[]
1,Though the indictment of a woman for her own p...,https://eu.usatoday.com/story/news/nation/2019...,usa-today,abortion,center,Non-biased,Somewhat factual but also opinionated,[]
2,Ingraham began the exchange by noting American...,https://www.breitbart.com/economy/2020/01/12/d...,breitbart,immigration,right,No agreement,No agreement,['flood']
3,The tragedy of America’s 18 years in Afghanist...,http://feedproxy.google.com/~r/breitbart/~3/ER...,breitbart,international-politics-and-world-news,right,Biased,Somewhat factual but also opinionated,"['tragedy', 'stubborn']"
4,The justices threw out a challenge from gun ri...,https://www.huffpost.com/entry/supreme-court-g...,msnbc,gun-control,left,Non-biased,Entirely factual,[]


In [19]:
# binarize classification problem
df_sg1 = df_sg1[df_sg1['Label_bias']!='No agreement']
df_sg1 = df_sg1[df_sg1['Label_bias'].isna()==False]
df_sg1.replace(to_replace='Biased', value=1, inplace=True)
df_sg1.replace(to_replace='Non-biased', value=0, inplace=True)

df_sg2 = df_sg2[df_sg2['Label_bias']!='No agreement']
df_sg2.replace(to_replace='Biased', value=1, inplace=True)
df_sg2.replace(to_replace='Non-biased', value=0, inplace=True)

# test pipeline set
df_sg1, exclude = train_test_split(df_sg1, test_size=0.95)
df_sg2, exclude = train_test_split(df_sg2, test_size=0.95)

In [20]:
# Stratified k-Fold instance
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [30]:
# helper functions called in skfold loop

def pd_to_tf(df):
    """convert a pandas dataframe into a tensorflow dataset"""
    target = df.pop('Label_bias')
    sentence = df.pop('sentence')
    return tf.data.Dataset.from_tensor_slices((sentence.values, target.values))

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])
    plt.show()

def tokenize(df, model_name):
    """convert a pandas dataframe into a tensorflow dataset and run hugging face's tokenizer on data"""
    df2 = df.copy(deep=False)
    target = df2.pop('Label_bias')
    sentence = df2.pop('sentence')
    
    if model_name=='bert':
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    elif model_name=='roberta':
        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    elif model_name=='deberta':
        tokenizer = DebertaTokenizer.from_pretrained("kamalkraj/deberta-base")
    elif model_name=='distilbert':
        tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    elif model_name=='electra':
        tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
    elif model_name=='xlnet':
        tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

    train_encodings = tokenizer(
                        sentence.tolist(),                      
                        add_special_tokens = True, # add [CLS], [SEP]
                        truncation = True, # cut off at max length of the text that can go to BERT
                        padding = True, # add [PAD] tokens
                        return_attention_mask = True, # add attention mask to not focus on pad tokens
              )
    
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(train_encodings), 
         target.tolist()))
    return dataset

In [44]:
def run_model_5fold(df_name, df_train, model_name, freeze_encoder=False, pretrained=False, plot=False):
    """"freeze flags whether encoder layer should be frozen to not destroy transfer learning. Only set to false when enough data is provided"""

    # these variables will be needed for skfold to select indices
    Y = df_train['Label_bias']
    X = df_train['sentence']

    # hyperparams
    BUFFER_SIZE = 10000
    BATCH_SIZE = 24
    k = 1

    val_loss = []
    val_acc = []
    val_prec = []
    val_rec = []
    val_f1 = []
    val_f1_micro = []
    val_f1_wmacro = []
    
    if pretrained==True:
        optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
        
        if model_name=='bert':
            transfer_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
            
        elif model_name=='roberta':
            transfer_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
            
        elif model_name=='deberta':
            transfer_model = TFDebertaForSequenceClassification.from_pretrained("kamalkraj/deberta-base")
            
        transfer_model.compile(optimizer=optimizer, loss='binary_crossentropy') 
        transfer_model.load_weights(f'./checkpoints/{model_name}_final_checkpoint_news_headlines_USA')
        trained_model_layer = transfer_model.get_layer(index=0).get_weights()
            

    for train_index, val_index in skfold.split(X,Y):
        print('### Start fold {}'.format(k))

        # split into train and validation set
        train_dataset = df_train.iloc[train_index]
        val_dataset = df_train.iloc[val_index]

        # prepare data for transformer
        train_dataset = tokenize(train_dataset, model_name)
        val_dataset = tokenize(val_dataset, model_name)

        # mini-batch it
        train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
        val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

        # create new model
        if model_name == 'bert':
            model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
        if model_name == 'distilbert':
            model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
        elif model_name == 'roberta':
            model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
        elif model_name == 'electra':
            model = TFElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator')
        elif model_name == 'xlnet':
            model = TFXLNetForSequenceClassification.from_pretrained('xlnet-base-cased')
        elif model_name == 'deberta':
            model = TFDebertaForSequenceClassification.from_pretrained("kamalkraj/deberta-base")


        if freeze_encoder == True:
            for w in model.get_layer(index=0).weights:
                w._trainable = False

        # compile it
        optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
        model.compile(optimizer=optimizer, loss='binary_crossentropy') 

        # transfer learning
        if pretrained == True:
            model.get_layer(index=0).set_weights(trained_model_layer) # load bias-specific weights

        # after 2 epochs without improvement, stop training
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

        # fit it
        history = model.fit(train_dataset, epochs=1, validation_data = val_dataset, callbacks=[callback])

        # plot history
        if plot:
            plot_graphs(history,'loss')

        # evaluate
        loss = model.evaluate(val_dataset)

        if model_name == 'xlnet':
            yhats = []
            for row in df_train.iloc[val_index]['sentence']:
                input = tokenizer(row, return_tensors="tf")
                output = model(input)
                logits = output.logits.numpy()[0]
                candidates = logits.tolist()
                decision = candidates.index(max(candidates))
                yhats.append(decision)
        else:
            logits = model.predict(val_dataset)  
            yhats = []
            for i in logits[0]:
                # assign class label according to highest logit
                candidates = i.tolist()
                decision = candidates.index(max(candidates))
                yhats.append(decision)

        y = []
        for text, label in val_dataset.unbatch():   
            y.append(label.numpy())

        val_loss.append(loss)
        val_acc.append(accuracy_score(y, yhats))
        val_prec.append(precision_score(y, yhats))
        val_rec.append(recall_score(y, yhats))
        val_f1.append(f1_score(y, yhats))
        val_f1_micro.append(f1_score(y, yhats, average='micro'))
        val_f1_wmacro.append(f1_score(y, yhats, average='weighted'))

        tf.keras.backend.clear_session()

        k += 1

        return {'loss': val_loss, 'acc': val_acc, 'prec': val_prec, 'rec': val_rec, 'f1': val_f1, 
                'f1_micro': val_f1_micro, 'f1_wmacro': val_f1_wmacro, 'model_name': model_name, 
                'distant': pretrained, 'df_name': df_name} 

In [45]:
def measure(d, results):
    loss_cv = np.mean(d['loss'])
    acc_cv = np.mean(d['acc'])
    prec_cv = np.mean(d['prec'])
    rec_cv = np.mean(d['rec'])
    f1_cv = np.mean(d['f1'])
    f1_micro_cv = np.mean(d['f1_micro'])
    f1_wmacro_cv = np.mean(d['f1_wmacro'])
    
    row = {
        'Dataset': d['df_name'],
        'Model': d['model_name'], 
        'Distant': d['distant'], 
        'Loss': loss_cv, 
        'Accuracy': acc_cv, 
        'Precision': prec_cv, 
        'Recall': rec_cv, 
        'F1': f1_cv, 
        'F1 Micro': f1_micro_cv, 
        'F1 Weighted': f1_wmacro_cv
    }
    
    results = results.append(row, ignore_index=True)
    print(row)
    return results


In [46]:
# run all experiments
models = [
    {'name': 'bert', 'distant': False}, 
    {'name': 'bert', 'distant': True},
    {'name': 'roberta', 'distant': False}, 
    {'name': 'roberta', 'distant': True},
    {'name': 'distilbert', 'distant': False},
    {'name': 'xlnet', 'distant': False},
    {'name': 'electra', 'distant': False},
    {'name': 'deberta', 'distant': False},
    {'name': 'deberta', 'distant': True}
    
]
datasets = [
    {'name': 'sg1', 'data': df_sg1},
    {'name': 'sg2', 'data': df_sg2}
]

columns = ['Dataset', 'Model', 'Distant', 'Loss', 'Accuracy', 'Precision', 'Recall', 'F1', 'F1 Micro', 'F1 Weighted']
results = pd.DataFrame(columns=columns)

for df in datasets:
    for model in models:
        results = measure(run_model_5fold(df['name'], df['data'], model['name'], freeze_encoder=False, pretrained=model['distant'], plot=False), results)

### Start fold 1


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1/1 [==============================] - 3s 3s/step
{'Dataset': 'sg1', 'Model': 'bert', 'Distant': False, 'Loss': 3.426753282546997, 'Accuracy': 0.5, 'Precision': 0.5, 'Recall': 0.875, 'F1': 0.6363636363636364, 'F1 Micro': 0.5, 'F1 Weighted': 0.4181818181818182}


/tmp/ipykernel_25464/70255357.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(row, ignore_index=True)
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Start fold 1


...

1/1 [==============================] - 3s 3s/step
{'Dataset': 'sg1', 'Model': 'roberta', 'Distant': False, 'Loss': 0.9692485928535461, 'Accuracy': 0.5, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'F1 Micro': 0.5, 'F1 Weighted': 0.3333333333333333}


/home/jstil/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_25464/70255357.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(row, ignore_index=True)
/tmp/ipykernel_25464/70255357.py:23: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results = results.append(row, ignore_index=True)
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly

### Start fold 1


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2022-10-28 16:45:37.007358: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.75MiB (rounded to 7077888)requested by op tf_roberta_for_sequence_classification_1/roberta/encoder/layer_._10/attention/output/dropout_69/dropout/Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-10-28 16:45:37.012810: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2022-10-28 16:45:37.01282

ResourceExhaustedError: Graph execution error:

Detected at node 'tf_roberta_for_sequence_classification_1/roberta/encoder/layer_._10/attention/output/dropout_69/dropout/Mul' defined at (most recent call last):
    File "/home/jstil/miniconda3/envs/tf/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/jstil/miniconda3/envs/tf/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/jstil/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/jstil/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/jstil/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/jstil/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.

In [ ]:
print(results)